In [1]:

import cdt
import os
import pandas as pd
import numpy as np
from cdt.data import load_dataset
import networkx as nx
from cdt.causality.graph import SAM
import matplotlib.pyplot as plt
import seaborn as sns

Detecting 1 CUDA device(s).


In [2]:
data_fn = '211205wave2e.csv'

df = pd.read_csv(data_fn)

cols = df.columns

df=(df-df.mean())/df.std()

In [3]:
df.columns

Index(['W2_Age_year', 'W2_Gender', 'W2_Relationship', 'W2_Keyworker',
       'W2_Adults_household', 'W2_Children_household', 'W2_Change_Income',
       'W2_Chronic_illness_self', 'W2_Pregnant', 'W2_COVID19_anxiety',
       'W2_RISK_1month', 'W2_Risk_total', 'W2_Hygiene_total', 'W2_Dep_Total',
       'W2_GAD_Total', 'W2_Loneliness_Total', 'W2_Attach_Style'],
      dtype='object')

In [4]:
df.shape

(1325, 17)

In [17]:
# specify skeleton (constraints)
skeleton = 1- np.eye((17),dtype=np.float32)
no_effect_cols = [0, 1,]  # [AGE, Gender] can't be effects (set to 0)
skeleton[:, no_effect_cols] = 0
skeleton[6, :9] = 0  # change in income can't affect vars 0:8.
print(skeleton) 

[[0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1.]
 [0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.]
 [0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.]
 [0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]]


In [18]:
'''lr (float) – Learning rate of the generators
dlr (float) – Learning rate of the discriminator
mixed_data (bool) – Experimental – Enable for mixed-type datasets
lambda1 (float) – L0 penalization coefficient on the causal filters
lambda2 (float) – L2 penalization coefficient on the weights of the neural network
nh (int) – Number of hidden units in the generators’ hidden layers (regularized with lambda2)
dnh (int) – Number of hidden units in the discriminator’s hidden layers
train_epochs (int) – Number of training epochs
test_epochs (int) – Number of test epochs (saving and averaging the causal filters)
batch_size (int) – Size of the batches to be fed to the SAM model Defaults to full-batch
losstype (str) – type of the loss to be used (either ‘fgan’ (default), ‘gan’ or ‘mse’)
dagloss (bool) – Activate the DAG with No-TEARS constraint
dagstart (float) – Controls when the DAG constraint is to be introduced in the training (float ranging from 0 to 1, 0 denotes the start of the training and 1 the end)
dagpenalisation (float) – Initial value of the DAG constraint
dagpenalisation_increase (float) – Increase incrementally at each epoch the coefficient of the constraint
functional_complexity (str) – Type of functional complexity penalization (choose between ‘l2_norm’ and ‘n_hidden_units’)
hlayers (int) – Defines the number of hidden layers in the generators
dhlayers (int) – Defines the number of hidden layers in the discriminator
sampling_type (str) – Type of sampling used in the structural gates of the model (choose between ‘sigmoid’, ‘sigmoid_proba’ and ‘gumble_proba’)
linear (bool) – If true, all generators are set to be linear generators
nruns (int) – Number of runs to be made for causal estimation Recommended: >=32 for optimal performance
njobs (int) – Numbers of jobs to be run in Parallel Recommended: 1 if no GPU available, 2*number of GPUs else
gpus (int) – Number of available GPUs for the algorithm
verbose (bool) – verbose mode
'''

lrs = [0.01, 0.001]
daglosses = [True, False]
dagpenalizations = [0.01, 0.05, 0.1]
num_runs = 50



'''skeleton (numpy.ndarray) – 
A priori knowledge about the causal relationships as an adjacency matrix. 
Can be fed either directed or undirected links.
'''


for lr in lrs:
    for dagloss in daglosses:
        for dagpenalization in dagpenalizations:
            print(dagpenalization)
            
            settings = 'lr_' + str(lr) + '_dagloss_' + str(dagloss) + '_dagpen_' + str(dagpenalization) + '_numruns_' + str(num_runs)
            
            obj = SAM(lr=lr, dlr=0.001, mixed_data=True, lambda1=10, lambda2=0.001, nh=20,
                       dnh=200, train_epochs=3000, test_epochs=800, batch_size=- 1, losstype='fgan',
                       dagloss=dagloss, dagstart=0.5, dagpenalization=0, dagpenalization_increase=dagpenalization, 
                        hlayers=2, dhlayers=2, sampling_type='sigmoidproba',
                       linear=False, nruns=num_runs, njobs=1, gpus=1, verbose=None)

            output = obj.predict(df, graph=skeleton.T) 
            
            nx.write_gml(output, "graphs/wave_2_constrained_1var_attachment_{}.gml".format(settings))

            
       
    

0.01


100%|██████████| 3800/3800 [03:37<00:00, 17.43it/s, disc=1.04e+12, gen=-1.04e+11, regul_loss=0.574, tot=-1.77e+13]


0.05


100%|██████████| 3800/3800 [03:16<00:00, 19.34it/s, disc=1.39e+24, gen=-1.39e+23, regul_loss=0.846, tot=-2.36e+25]


0.1


...

100%|██████████| 3800/3800 [01:12<00:00, 52.63it/s, disc=1.77e+4, gen=-1.76e+3, regul_loss=1.48, tot=-2.99e+5]


0.05


100%|██████████| 3800/3800 [01:15<00:00, 50.14it/s, disc=1.44e+11, gen=-1.44e+10, regul_loss=1.47, tot=-2.45e+12]


0.1


100%|██████████| 3800/3800 [01:11<00:00, 53.43it/s, disc=-.331, gen=-.11, regul_loss=1.36, tot=-17.3]   
